## Evaluate Microsoft Phi 2 LLM for answer generation

https://huggingface.co/microsoft/phi-2

In [ ]:
try:
    import bert_score
except ImportError:
    !pip install bert_score

try:
    from evaluate import load
except ImportError:
    !pip install evaluate

try:
    import rouge_score
except ImportError:
    !pip install rouge_score

try:
    import ragas
except ImportError:
    !pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import json
import bert_score
import numpy as np
import re
from evaluate import load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# provide project root path
#ProjectRoot = "/content/drive/MyDrive/UMich Capstone/NoteBooks/"
ProjectRoot = "/MyDrive/UMich Capstone/NoteBooks/"
DatasetRoot = ProjectRoot + "Dataset/"

In [ ]:
# load context and question train set which was created by doc2query
train_df = pd.read_csv(DatasetRoot + 'q_a_trainset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/MyDrive/UMich Capstone/NoteBooks/Dataset/q_a_trainset.csv'

In [ ]:
# loading full article from json file
with open(DatasetRoot + 'raw_knowledge.json', 'r') as f:
    raw_text_json = json.load(f)

In [ ]:
raw_df = pd.DataFrame(list(raw_text_json.items()), columns=['raw_para_id', 'raw_text'])
raw_df['raw_para_id'] = raw_df['raw_para_id'].astype('int64')

In [ ]:
# create dataframe of raw, summarized paragraphs and question
train_df = train_df.merge(raw_df, left_on='raw_para_id', right_on='raw_para_id', how='left')

### Evaluation

In [ ]:
if torch.cuda.is_available():
    torch.set_default_device("cuda")
    print("CUDA is available!!")
else:
    raise RuntimeError("CUDA is not available!! LLM cannot run, rerun with GPU")

In [ ]:
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Prompt Engineering

In [ ]:
def generate_prompt_fewshot(context, question):
    prompt_template = """
You are an expert in understanding and interpreting provided text contexts. Given a context and a question, your task is to generate an accurate and informative answer based on the provided context. Here is the structure:

1. **Context:** The detailed text or passage that contains the information needed to answer the question.
2. **Question:** A specific question that needs to be answered based on the context.

Please make sure your response is clear, concise, and directly addresses the question. If the context does not contain sufficient information to answer the question, say I don't know.

**Example:**

**Context:**
"The rainforests of the Amazon are home to a vast diversity of species, including numerous plants, animals, and insects. These forests play a crucial role in regulating the Earth's climate by absorbing carbon dioxide and releasing oxygen. However, deforestation poses a significant threat to these ecosystems, leading to loss of habitat and biodiversity."

**Question:**
"Why are the rainforests of the Amazon important for the Earth's climate?"

**Answer:**
"The rainforests of the Amazon are important for the Earth's climate because they absorb carbon dioxide and release oxygen, helping to regulate the climate."

Please follow this format for each question:

**Context:**
{context}

**Question:**
{question}

**Final Answer:**
"""
    return prompt_template.format(context=context, question=question)



In [ ]:
def generate_prompt_zeroshot(context, query):
    prompt = f"""Give the answer to the user query delimited by triple backticks ```{query}```
                using the information given in context delimited by triple backticks ```{context}```.
                If there is no relevant information in the provided context, tell user that you did not have any relevant context to base your answer on. Be concise and output the answer.
                """
    return prompt

In [ ]:
def generate_prompt_zeroshot_2(context, question):
    prompt_template = f"""
Given the following context, please answer the question:

Context:
{context}

Question:
{question}
"""
    return prompt_template

In [ ]:
def generate_prompt_zeroshot_3(context, question):
    prompt = f"""
    You are an expert in understanding and interpreting provided text contexts. Given a context and a question, your task is to generate an accurate and informative answer based on the provided context. Here is the structure:

    **Context:**
    "{context}"

    **Question:**
    "{question}"

    **Answer:**
    """
    return prompt

#### Calculate Different metric scores

In [ ]:
# LLM inference wrapper
def AskLLM(context, question):
    prompt = generate_prompt_fewshot(context, question)

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

    # Ask LLM to answer
    outputs = model.generate(**inputs, max_length=500)
    text = tokenizer.batch_decode(outputs)[0]

    # Regular expression to extract the Final Answer
    answer_pattern = r"\*\*Final Answer:\*\*\s*(.*)<|endoftext|>"

    # Extract answer from LLM response
    match = re.search(answer_pattern, text, re.DOTALL)
    final_answer = ""
    if match:
        final_answer = match.group(1).strip()
        # print("Final Answer:", final_answer)
    else:
        print("Final Answer not found !!!")

    return final_answer

In [ ]:
candidate_answers = []
true_answers = []

for _, eval_data in train_df.iterrows():
    context = eval_data.raw_text
    question = eval_data.question

    true_answers.append(eval_data.paragraph)
    candidate_answers.append(AskLLM(context, question))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Final Answer not found !!!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Final Answer not found !!!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Calculate BERTScore
# bert_metrics = bert_score.score(cands=candidate_answers, refs=true_answers, model_type='roberta-large', nthreads=4)
bert_metrics = bert_score.score(cands=candidate_answers, refs=true_answers, model_type='bert-base-uncased', nthreads=4)

# Fetch precision, recall, F1 score from BERT score (https://lightning.ai/docs/torchmetrics/stable/text/bert_score.html)
print(f"Mean Precision: {np.mean(np.array(bert_metrics[0]))}")
print(f"Mean Recall: {np.mean(np.array(bert_metrics[1]))}")
print(f"Mean F1 Score: {np.mean(np.array(bert_metrics[2]))}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean Precision: 0.772310733795166
Mean Recall: 0.7730169296264648
Mean F1 Score: 0.7723470330238342


In [ ]:
# Calculate BERTScore via https://huggingface.co/spaces/evaluate-metric/bertscore
bertscore = load("bertscore")
bert_metrics2 = bertscore.compute(predictions=candidate_answers, references=true_answers, lang="en")

print(f"Mean Precision: {np.mean(np.array(bert_metrics2['precision']))}")
print(f"Mean Recall: {np.mean(np.array(bert_metrics2['recall']))}")
print(f"Mean F1 Score: {np.mean(np.array(bert_metrics2['f1']))}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean Precision: 0.7723107704271873
Mean Recall: 0.7730169172088305
Mean F1 Score: 0.7723470317820708


In [ ]:
# calculate meteor via https://huggingface.co/spaces/evaluate-metric/meteor
meteor = load('meteor')
meteor_score = meteor.compute(predictions=candidate_answers, references=true_answers)

print(f"METEOR Score: {meteor_score['meteor']}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR Score: 0.42784206605545555


In [ ]:
# calculate rouge via https://huggingface.co/spaces/evaluate-metric/rouge
rouge = load("rouge")
rouge_score = rouge.compute(predictions=candidate_answers, references=true_answers)

print(f"ROUGE Score: {rouge_score}")

ROUGE Score: {'rouge1': 0.4312733653918397, 'rouge2': 0.3210025761405472, 'rougeL': 0.39136625236961187, 'rougeLsum': 0.3899905976080511}
